<a href="https://www.kaggle.com/code/candacevogel/notebook97467baac6?scriptVersionId=262519770" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# CelebA Glasses Facial Characteristic Recognition

In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms.v2 as transforms
from torchvision.transforms import ToTensor
from torchvision import datasets

from os import path
from PIL import Image

# Load data - create path to files

In [2]:
dataset_root = '/kaggle/input/celeba-dataset'
# input data from dataset

image_path = path.join(dataset_root, 'img_align_celeba/img_align_celeba')
# image_path contains the jpg files
attributes_path = path.join(dataset_root, 'list_attr_celeba.csv')
# file path for the csv file containing the attributes

# Read attributes CSV

In [3]:
df_attributes = pd.read_csv(attributes_path)
# dataframe (df) is a type of table that panda library uses
# read csv file using pandas library ^
df_attributes.replace(-1, 0, inplace=True)
# 
df_attributes['Eyeglasses'].head()

# this output will print just the eyeglasses data

0    0
1    0
2    0
3    0
4    0
Name: Eyeglasses, dtype: int64

In [4]:
# Replace -1 with 0

In [5]:
df_attributes.replace(-1, 0, inplace=True)
df_attributes.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


# How many people are wearing glasses?

In [6]:
df_attributes['Eyeglasses'].value_counts()
# prints number of people wearing glasses in dataset

Eyeglasses
0    189406
1     13193
Name: count, dtype: int64

In [7]:
# Balance dataset